In [0]:
paises = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/frodriguez@insulet.com/pai_ses-2.csv")

In [0]:
from pyspark.sql.functions import col

paises = paises.withColumn("total_cases_per_million", col("total_cases_per_million").cast("double"))
paises = paises.withColumn("total_deaths_per_million", col("total_deaths_per_million").cast("double"))
paises = paises.withColumn("diabetes_prevalence", col("diabetes_prevalence").cast("double"))
paises = paises.withColumn("hdi", col("hdi").cast("double"))
# casting strings for doubles

paises

Out[2]: DataFrame[code: string, continent: string, country: string, date: string, total_cases_per_million: double, total_deaths_per_million: double, population_density: string, cardiovasc_death_rate: string, diabetes_prevalence: double, life_expectancy: string, hdi: double]

In [0]:
paises = paises.na.drop()

## Train/Test Split

![](https://files.training.databricks.com/images/301/TrainTestSplit.png)

**Question**: Why is it necessary to set a seed? What happens if I change my cluster configuration?

In [0]:
trainDF, testDF = paises.randomSplit([.8, .2], seed=42)
print(trainDF.cache().count())

147

Let's change the # of partitions (to simulate a different cluster configuration), and see if we get the same number of data points in our training set.

In [0]:
trainRepartitionDF, testRepartitionDF = (paises
                                         .repartition(24)
                                         .randomSplit([.8, .2], seed=42))

print(trainRepartitionDF.count())

# we just simulated a different cluster
# despite same seed 42, we have 158 rows instead of 165 (previous cell) 

146

In [0]:
display(trainDF.select("total_cases_per_million", "hdi").summary())

summary,total_cases_per_million,hdi
count,147,147
mean,62302.86707482991,0.7251632653061223
stddev,63405.76922256815,0.1522297309006577
min,68.961,0.394
25%,7099.542,0.594
50%,48589.891,0.75
75%,99247.528,0.851
max,253504.891,0.957


In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
from pyspark.ml.feature import VectorAssembler

# we want to output the values of chosen columns into a single column, as a vector, and we will call it features
vecAssembler = VectorAssembler(inputCols=["hdi"], outputCol="features")

# transform takes a df and changes columns, or appends columns
# combining column values into a single column
vecTrainDF = vecAssembler.transform(trainDF)

In [0]:
lr = LinearRegression(featuresCol="features", labelCol="total_cases_per_million")
lrModel = lr.fit(vecTrainDF)

## Inspect the model

In [0]:
m = lrModel.coefficients[0]
b = lrModel.intercept

print(f"The formula for the linear regression line is y = {m:.2f}x + {b:.2f}")

# En este caso nuestra x es el valor de HDI
# si un pais tiene 0.5, esperariamos -1271.55
# si tuviera 0.9, esperariamos 111,667.68

The formula for the linear regression line is y = 282348.08x + -142445.59

## Apply model to test set

In [0]:
vecTestDF = vecAssembler.transform(testDF)

predDF = lrModel.transform(vecTestDF)

predDF.select("hdi", "features", "total_cases_per_million", "prediction").show()

+-----+--------+-----------------------+-------------------+
 hdi|features|total_cases_per_million| prediction|
+-----+--------+-----------------------+-------------------+
0.795| [0.795]| 70611.786| 82021.13528007633|
0.776| [0.776]| 115145.978| 76656.5216917992|
0.944| [0.944]| 8749.151| 124090.99973551271|
0.545| [0.545]| 1999.593| 11434.11438169307|
 0.78| [0.78]| 85942.247| 77785.91402617333|
0.765| [0.765]| 103634.293| 73550.69277227033|
0.929| [0.929]| 48107.208| 119855.77848160977|
 0.48| [0.48]| 640.932| -6918.511051886599|
0.524| [0.524]| 13484.375| 5504.80462622887|
0.742| [0.742]| 86016.738| 67056.68684961906|
 0.94| [0.94]| 93265.583| 122961.60740113858|
0.748| [0.748]| 4756.612| 68750.77535118026|
0.707| [0.707]| 3516.592| 57174.50392384539|
0.485| [0.485]| 3161.884| -5506.770633918932|
0.611| [0.611]| 4136.06| 30069.087898866244|
0.663| [0.663]| 34045.616| 44751.18824572995|
0.955| [0.955]| 122590.16| 127196.82865504158|
0.916| [0.916]| 146594.12| 116185.2533948938|
0.737| [0.737]| 115614.117| 65644.94643165139|
0.483| [0.483]| 3160.068|-6071.4668011059985|
+-----+--------+-----------------------+-------------------+
only showing top 20 rows

## Evaluate Model

Let's see how our linear regression model with just one variable does. Does it beat our baseline model?

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="total_cases_per_million", metricName="rmse")

rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE is {rmse}")

r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF)
print(f"r2 is {r2}")

RMSE is 35007.88726699643
r2 is 0.3749156266340665

El error es muy alto, pero la r2 mejoro

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>